In [99]:
import pandas as pd
YEAR = 2004
races = pd.read_csv('.\\data\\races.csv')
raceResults = pd.read_csv('.\\data\\results.csv')
lapTimes = pd.read_csv('.\\data\\lapTimes.csv')
driverStandings = pd.read_csv('.\\data\\driverStandings.csv')

# złączanie "tabel" results oraz races po kolumnie raceId
merged_data = pd.merge(raceResults, races[['raceId', 'year']], on='raceId')

# usunięcie niekluczowych danych
merged_data.drop(["constructorId", "number", "positionText", 'position', 'points', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId'], axis=1, inplace=True)

# tabela zbiorcza
all_season_stats = []

# sprawdzenie, który z kierowców był mistrzem w danym sezonie
last_races = races.groupby('year')['raceId'].max().reset_index()
last_race_standings = pd.merge(last_races, driverStandings, on='raceId')
champions = last_race_standings[last_race_standings['position'] == 1][['year', 'driverId']]

# champions.to_csv(".\\data\\champions.csv", encoding='utf-8', index=False)

# iterowanie po wszystkich latach
for year in range(1950, 2018):
    # print(f"\n{year}")
    year_data = merged_data[merged_data['year'] == year]

    number_of_races = year_data.nunique()["raceId"]

    # liczenie średniej kolumn positionOrder oraz grid grupując po kolumnie driverId
    season_stats = year_data.groupby('driverId').agg({
        'positionOrder': 'mean',
        'grid': 'mean'
    }).reset_index()

    season_stats.rename(columns={'positionOrder': 'avg_racePosition', 'grid': 'avg_gridPosition'}, inplace=True)

    wins_percentage = year_data.groupby('driverId')['positionOrder'].apply(lambda x: (x == 1).sum() / number_of_races * 100)
    season_stats['winsPercentage'] = season_stats['driverId'].map(wins_percentage)

    # obliczanie % przejechanych okrążeń w sezonie
    year_lap_times = lapTimes[lapTimes['raceId'].isin(year_data['raceId'])]
    max_laps = year_lap_times.groupby('raceId')['lap'].max()
    driver_laps = year_lap_times.groupby(['raceId', 'driverId'])['lap'].max()
    completed_lap_percentage = driver_laps / driver_laps.index.get_level_values(0).map(max_laps)
    completed_lap_percentage = completed_lap_percentage.groupby('driverId').mean() * 100
    season_stats['completedLapPercentage'] = season_stats['driverId'].map(completed_lap_percentage)

    # dodanie kolumny określająca, czy dany kierowca został mistrzem w tym sezonie
    season_stats['championshipWinner'] = season_stats['driverId'].apply(lambda x: 1 if ((year, x) in zip(champions['year'], champions['driverId'])) else 0)

    # usuwanie wartości NaN - dane o przejechanych okrążeniach zbierane są dopiero od sezonu 1996
    season_stats.dropna(subset=['completedLapPercentage'], inplace=True)

    season_stats.drop(columns=['driverId'], inplace=True)
    # season_stats['year'] = year

    # dodanie danych do zbiorczej tabeli
    all_season_stats.append(season_stats)

# połączenie wszystkich danych z tabeli zbiorczej do jednej tabeli
merged_season_stats = pd.concat(all_season_stats, ignore_index=True)

merged_season_stats.to_csv(".\\data\\merged_season_stats.csv", encoding='utf-8', index=False)
